<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#L'obiettivo" data-toc-modified-id="L'obiettivo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>L'obiettivo</a></span></li><li><span><a href="#Importare-i-dati" data-toc-modified-id="Importare-i-dati-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Importare i dati</a></span></li><li><span><a href="#Preparazione-dei-dati" data-toc-modified-id="Preparazione-dei-dati-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preparazione dei dati</a></span><ul class="toc-item"><li><span><a href="#Missing-values" data-toc-modified-id="Missing-values-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Missing values</a></span></li><li><span><a href="#Data-types" data-toc-modified-id="Data-types-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Data types</a></span></li></ul></li><li><span><a href="#Separando-i-dati-per-la-validazione" data-toc-modified-id="Separando-i-dati-per-la-validazione-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Separando i dati per la validazione</a></span><ul class="toc-item"><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Xgboost" data-toc-modified-id="Xgboost-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Xgboost</a></span></li></ul></li><li><span><a href="#Interpretabilità" data-toc-modified-id="Interpretabilità-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Interpretabilità</a></span><ul class="toc-item"><li><span><a href="#Permutation-importance" data-toc-modified-id="Permutation-importance-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Permutation importance</a></span></li><li><span><a href="#Partial-plots" data-toc-modified-id="Partial-plots-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Partial plots</a></span></li><li><span><a href="#Shap-values" data-toc-modified-id="Shap-values-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Shap values</a></span></li></ul></li></ul></div>

# L'obiettivo

Come primo passo, cerco di controllare se ci siano cambiamenti sostanziali fra i dati per il 2018 ed il 2019. Più precisamente, se possa costruire un modello di classificazione dove le classi sono "Appartiene al 2018" o "Appartiene al 2019" ([adversarial validation](http://manishbarnwal.com/blog/2017/02/15/introduction_to_adversarial_validation/)).

Se riesco a costruire un *buon* modello predittivo, significa che ci sono variazioni significative fra i due anni. Normalmente, escluderei le variabili che sono rilevanti per distinguere il 2018 dal 2019 in un modello addestrato (trained) per prevedere il 2019. Lo posso fare senza conoscere la variabile che voglio stimare per il 2019. 

Ma in questo caso, voglio trovare quali variabili permettono di distinguere il 2018 dal 2019 come strumento per sinteticamente descrivere le differenze fra 2018 e 2019.

# Importare i dati

Come primo passo, importo i dati per il 2018 ed il 2019 per i fornitori. 

In [1]:
import pandas as pd


a2018 = pd.read_csv(
    "../input/fornitori2018.csv", skiprows=0, encoding="ISO-8859-1", decimal=","
)
print(a2018.shape)
a2018.head()

(110876, 12)


,#Partita_Iva,Ragione_Sociale,Forma_Societaria,Nazione_Sede_legale,Regione_Sede_legale,Provincia_Sede_legale,Comune_Sede_legale,Indirizzo_Sede_legale,Numero_Aggiudicazioni,Numero_Abilitazioni,Numero_Transazioni,Numero_Contratti_attivi
0,02455120226,\tSEA CONSULENZE E SERVIZI,Società a Responsabilità Limitata,ITALIA,TRENTINO ALTO ADIGE,TRENTO,TRENTO,VIA UNTERVEGER 52 - TRENTO,0,6,3,3
1,03475830711,ARPATEK SRL,Società a Responsabilità Limitata,ITALIA,PUGLIA,FOGGIA,FOGGIA,VIA MANFREDONIA KM 2.200,0,1,0,0
2,00688650217,CAPONE PIETRO S.R.L.,Società a Responsabilità Limitata,ITALIA,TRENTINO ALTO ADIGE,BOLZANO/BOZEN,BOLZANO/BOZEN,"VIA COPERNICO, 7",0,1,0,0
3,02903790802,F.LLI SACCO S.R.L.S.,Società a Responsabilità Limitata,ITALIA,CALABRIA,REGGIO DI CALABRIA,BENESTARE,"BENESTARE (RC) , C/DA BRUCA, 68",0,1,0,0
4,02303790808,SEGECO S.R.L.,Società a Responsabilità Limitata,ITALIA,CALABRIA,REGGIO DI CALABRIA,CAMPO CALABRO,"VIA SACERDOTE SCOPELLITI, 53",0,3,0,0


In [2]:
a2019 = pd.read_csv(
    "../input/fornitori2019.csv", skiprows=0, encoding="ISO-8859-1", decimal=","
)
print(a2019.shape)
a2019.head()

(137551, 14)


,#Partita_Iva,Ragione_Sociale,Forma_Societaria,Nazione_Sede_legale,Regione_Sede_legale,Provincia_Sede_legale,Comune_Sede_legale,Indirizzo_Sede_legale,Latitudine_Sede_legale,Longitudine_Sede_legale,Numero_Aggiudicazioni,Numero_Abilitazioni,Numero_Transazioni,Numero_Contratti_attivi
0,00688650217,CAPONE PIETRO S.R.L.,Società a Responsabilità Limitata,ITALIA,TRENTINO ALTO ADIGE,BOLZANO/BOZEN,BOLZANO/BOZEN,"VIA COPERNICO, 7",46.47142,11.32011,0,1,0,0
1,03024340808,ECO. GROUP S.R.L.,Società a Responsabilità Limitata,ITALIA,CALABRIA,REGGIO DI CALABRIA,ARDORE,"C/DA SERRA, SNC FRAZ. MARINA",38.17991,16.20984,0,9,0,0
2,02903790802,F.LLI SACCO S.R.L.S.,Società a Responsabilità Limitata,ITALIA,CALABRIA,REGGIO DI CALABRIA,BENESTARE,"BENESTARE (RC) , C/DA BRUCA, 68",38.18399,16.13998,0,1,3,3
3,02303790808,SEGECO S.R.L.,Società a Responsabilità Limitata,ITALIA,CALABRIA,REGGIO DI CALABRIA,CAMPO CALABRO,"VIA SACERDOTE SCOPELLITI, 53",38.21941,15.65679,0,3,0,0
4,02972900803,SOPHIA S.R.L.,Società a Responsabilità Limitata,ITALIA,CALABRIA,REGGIO DI CALABRIA,SANT'AGATA DEL BIANCO,"VIA MAZZINI, 27",38.09197,16.08343,0,3,0,0


I dati per il 2019 sono più numerosi, quindi se usassi come criterio l'accuratezza nel prevedere l'appartenenza al 2018/2019. Di conseguenza, uso l'AUC. 

Ma prima, la preparazione dei dati.

# Preparazione dei dati

In [3]:
a2018["year"] = 2018
a2018.columns

Index(['#Partita_Iva', 'Ragione_Sociale', 'Forma_Societaria',
       'Nazione_Sede_legale', 'Regione_Sede_legale', 'Provincia_Sede_legale',
       'Comune_Sede_legale', 'Indirizzo_Sede_legale', 'Numero_Aggiudicazioni',
       'Numero_Abilitazioni', 'Numero_Transazioni', 'Numero_Contratti_attivi',
       'year'],
      dtype='object')

In [4]:
a2019["year"] = 2019
a2019.columns

Index(['#Partita_Iva', 'Ragione_Sociale', 'Forma_Societaria',
       'Nazione_Sede_legale', 'Regione_Sede_legale', 'Provincia_Sede_legale',
       'Comune_Sede_legale', 'Indirizzo_Sede_legale', 'Latitudine_Sede_legale',
       'Longitudine_Sede_legale', 'Numero_Aggiudicazioni',
       'Numero_Abilitazioni', 'Numero_Transazioni', 'Numero_Contratti_attivi',
       'year'],
      dtype='object')

Considero solo le colonne in comune ai due data sets:

In [5]:
common = set(a2018.columns).intersection(set(a2019.columns))
print(common)

{'Numero_Transazioni', 'Indirizzo_Sede_legale', 'Forma_Societaria', 'Comune_Sede_legale', 'Ragione_Sociale', 'Numero_Aggiudicazioni', '#Partita_Iva', 'Provincia_Sede_legale', 'Numero_Abilitazioni', 'year', 'Nazione_Sede_legale', 'Regione_Sede_legale', 'Numero_Contratti_attivi'}


Creo un unico dataset:

In [6]:
df = pd.concat((a2018[common], a2019[common]), axis=0)
df.shape

(248427, 13)

In [7]:
set(df["Forma_Societaria"])

{'ASSOCIAZIONI/ENTI NON ONLUS',
 "AZIENDA CON PARTECIPAZIONE DI CAPITALE PUBBLICO e UNIVERSITA'",
 'Altro (Cooperativa Sociale, Fondazione, ...)',
 'Altro (Cooperativa Sociale, Fondazione, Associazioni, ...)',
 'Consorzio fra società cooperative di produzione e lavoro/Consorzio fra imprese artigiane',
 'Consorzio stabile',
 'Impresa Individuale',
 'ONLUS',
 'Professione non organizzata in ordini o collegi (cfr. L. 4/2013)',
 'Professione organizzata (cfr. Art. 2229 cc)',
 'Rete di imprese',
 'Società Cooperativa',
 'Società a Responsabilità Limitata',
 'Società costituita in altro Stato Membro UE',
 'Società costituita in altro Stato extra UE',
 'Società in Accomandita Semplice',
 'Società in Accomandita per azioni',
 'Società in Nome Collettivo',
 'Società per Azioni',
 'Società semplice',
 nan}

## Missing values

In [8]:
df.isnull().mean()

Numero_Transazioni         0.000000
Indirizzo_Sede_legale      0.000040
Forma_Societaria           0.000338
Comune_Sede_legale         0.000064
Ragione_Sociale            0.000000
Numero_Aggiudicazioni      0.000000
#Partita_Iva               0.000105
Provincia_Sede_legale      0.000064
Numero_Abilitazioni        0.000000
year                       0.000000
Nazione_Sede_legale        0.000032
Regione_Sede_legale        0.000064
Numero_Contratti_attivi    0.000000
dtype: float64

I dati mancanti sono per variabili categoriche. Rimpiazzo con un valore "Missing":

In [9]:
d = {
    "Provincia_Sede_legale": "Missing",
    "Nazione_Sede_legale": "Missing",
    "Comune_Sede_legale": "Missing",
    "Forma_Societaria": "Missing",
    "Regione_Sede_legale": "Missing",
    "#Partita_Iva": "Missing",
}
df.fillna("Missing", inplace=True)

In [10]:
df.isnull().sum()

Numero_Transazioni         0
Indirizzo_Sede_legale      0
Forma_Societaria           0
Comune_Sede_legale         0
Ragione_Sociale            0
Numero_Aggiudicazioni      0
#Partita_Iva               0
Provincia_Sede_legale      0
Numero_Abilitazioni        0
year                       0
Nazione_Sede_legale        0
Regione_Sede_legale        0
Numero_Contratti_attivi    0
dtype: int64

## Data types

Controllo il tipo delle colonne:

In [11]:
df.dtypes

Numero_Transazioni          int64
Indirizzo_Sede_legale      object
Forma_Societaria           object
Comune_Sede_legale         object
Ragione_Sociale            object
Numero_Aggiudicazioni       int64
#Partita_Iva               object
Provincia_Sede_legale      object
Numero_Abilitazioni         int64
year                        int64
Nazione_Sede_legale        object
Regione_Sede_legale        object
Numero_Contratti_attivi     int64
dtype: object

Le variabili di tipo ``object`` sono variabili categoriche. La versione che sto usando di ``pandas`` è la seguente:

In [12]:
pd.__version__

'1.0.3'

Quindi:

In [13]:
categorical_columns = [
    "#Partita_Iva",
    "Provincia_Sede_legale",
    "Regione_Sede_legale",
    "Ragione_Sociale",
    "Forma_Societaria",
    "Comune_Sede_legale",
    "Nazione_Sede_legale",
    "Indirizzo_Sede_legale"
]

for i in categorical_columns:
    df[i] = df[i].astype("category")

df.dtypes

Numero_Transazioni            int64
Indirizzo_Sede_legale      category
Forma_Societaria           category
Comune_Sede_legale         category
Ragione_Sociale            category
Numero_Aggiudicazioni         int64
#Partita_Iva               category
Provincia_Sede_legale      category
Numero_Abilitazioni           int64
year                          int64
Nazione_Sede_legale        category
Regione_Sede_legale        category
Numero_Contratti_attivi       int64
dtype: object

Quanti valori distinti ci sono per colonna?

In [14]:
df.nunique()

Numero_Transazioni            653
Indirizzo_Sede_legale      141287
Forma_Societaria               21
Comune_Sede_legale           6900
Ragione_Sociale            148200
Numero_Aggiudicazioni          13
#Partita_Iva               149213
Provincia_Sede_legale         112
Numero_Abilitazioni            72
year                            2
Nazione_Sede_legale            32
Regione_Sede_legale            22
Numero_Contratti_attivi       746
dtype: int64

In [15]:
df = df.sample(df.shape[0], random_state=2020)

Ho fissato il random state per assicurare la riproducibilità dei calcoli (a parità di versione delle librerie usate).

In [16]:
df.head(n=10)

,Numero_Transazioni,Indirizzo_Sede_legale,Forma_Societaria,Comune_Sede_legale,Ragione_Sociale,Numero_Aggiudicazioni,#Partita_Iva,Provincia_Sede_legale,Numero_Abilitazioni,year,Nazione_Sede_legale,Regione_Sede_legale,Numero_Contratti_attivi
135713,0,"PIAZZA IV MARTIRI, 33",Società a Responsabilità Limitata,CARMAGNOLA,VM SERVICE S.R.L.,0,10169750014,TORINO,3,2019,ITALIA,PIEMONTE,0
108481,3,VIA DI CASTEL ROMANO 100,Società per Azioni,ROMA,RINA CONSULTING - CENTRO SVILUPPO MATERIALI SPA,0,00903541001,ROMA,2,2019,ITALIA,LAZIO,3
3993,2,"VIA VITTORIO EMANUELE II, 51",Società a Responsabilità Limitata,CONSELVE,ALBA,0,04563110289,PADOVA,6,2019,ITALIA,VENETO,2
76277,13,VIALE WALTER GHELFI 24,Società a Responsabilità Limitata,RIMINI,NOVADYS ITALIA,0,02441440407,RIMINI,1,2018,ITALIA,EMILIA ROMAGNA,13
77420,3,VIA UMBERTO GIORDANO 7,Società in Accomandita Semplice,CITTA' DI CASTELLO,OFFICINA DEL TRATTORE SAS DI TIRIMBELLI ROBERT...,0,00176050540,PERUGIA,4,2018,ITALIA,UMBRIA,3
43796,0,VIA PROVINCIALE VECCHIA N° 11,Società in Nome Collettivo,SESTOLA,EDILTER DI MAMMI MARIO & C. S.N.C.,0,01367780366,MODENA,5,2019,ITALIA,EMILIA ROMAGNA,0
95360,0,"VIA MAESTRI DEL LAVORO, 24/B",Società a Responsabilità Limitata,TERNI,NOVEDIL S.R.L.,0,00387050552,TERNI,4,2019,ITALIA,UMBRIA,1
8049,4,VIA TREVISO 51,Società in Accomandita Semplice,SILEA,ARTUFFICIO,0,04056590260,TREVISO,1,2018,ITALIA,VENETO,4
69926,2,"STRADELLA LOBBIA, 2",Impresa Individuale,VERONA,MAZZAI ADRIANO,0,02182910238,VERONA,1,2018,ITALIA,VENETO,2
101764,0,"VIA ESTRAMURALE A LEVANTE, 86",Impresa Individuale,PUTIGNANO,TECNO GARDEN DI PATRUNO DONATO,0,07299500723,BARI,6,2018,ITALIA,PUGLIA,0


# Separando i dati per la validazione

Utilizzo una cross validation con $k=10$ sottoinsiemi.

In [17]:
import sklearn
import category_encoders

print(sklearn.__version__)
print(category_encoders.__version__)

0.22.2.post1
2.1.0


Voglio prevedere a quale anno i dati fanno riferimento. Separo ``#Anno_Riferiemnto`` dai dati:

In [18]:
Y = df["year"]
X = df.drop(["year"], axis=1)

Converto $Y$ a 0/1:

In [19]:
Y = Y.map({2018: "0", 2019: "1"})

## Random Forest

Iterando sul numero di fold:

In [20]:
from category_encoders import CatBoostEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score



kf = KFold(n_splits=10, random_state=2020, shuffle=True)
auc_score = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    encoder = CatBoostEncoder(cols=categorical_columns)
    encoder.fit(X_train, Y_train)
    X_train = encoder.transform(X_train)
    X_test = encoder.transform(X_test)

    model = RandomForestClassifier(random_state=2020).fit(X_train, Y_train)
    auc_score += [roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1])]

In [21]:
import numpy as np


print(np.mean(auc_score), "+/-", np.std(auc_score))

0.3352418099660221 +/- 0.002449607855333874


## Xgboost

In [22]:
from xgboost import XGBClassifier


auc_score = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

    encoder = CatBoostEncoder(cols=categorical_columns)
    encoder.fit(X_train, Y_train)
    X_train = encoder.transform(X_train)
    X_test = encoder.transform(X_test)

    model = XGBClassifier(random_state=2020).fit(X_train, Y_train)
    auc_score += [roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1])]
    
print(np.mean(auc_score), "+/-", np.std(auc_score))

0.4267767115213418 +/- 0.0030103837832385063


Questo senza ottimizzare gli iper-parametri. Significa che conoscere i dati per il 2018 non aiuta molto a prevedere i dati per il 2019.

# Interpretabilità

Faccio un trading su tutti i dati:

In [23]:
encoder = CatBoostEncoder(cols=categorical_columns)
encoder.fit(X, Y)
X = encoder.transform(X)


model = XGBClassifier(random_state=2020).fit(X, Y)

## Permutation importance

Come riferimento per l'explainability del modello (per una spiegazione dettagliata) considero il [corso di Kaggle](https://www.kaggle.com/dansbecker/permutation-importance).

Per valutare límportanza delle singole variabili si può testare quanto migliori siano le variabili originalu rispetto a delle permutazioni casuali. Se permutare le variabili non danneggia significativamente le performance del modello (o addirittura le migliora) le variabili non sono rilevanti. Nella prossima tabella viene riportata la differenza in performance dovuta alla permutazione casuale delle variabili:

In [24]:
import eli5


eli5.__version__

/home/marco/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/marco/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.
/home/marco/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) o

'0.10.1'

In [25]:
from eli5.sklearn import PermutationImportance


perm = PermutationImportance(model, random_state=2020).fit(X, Y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
0.0836 ± 0.0017,Numero_Contratti_attivi
0.0578 ± 0.0006,Numero_Transazioni
0.0545 ± 0.0004,#Partita_Iva
0.0193 ± 0.0008,Forma_Societaria
0.0191 ± 0.0006,Indirizzo_Sede_legale
0.0139 ± 0.0009,Ragione_Sociale
0.0138 ± 0.0008,Comune_Sede_legale
0.0099 ± 0.0009,Numero_Abilitazioni
0.0043 ± 0.0005,Provincia_Sede_legale
0.0033 ± 0.0006,Regione_Sede_legale


In [26]:
X.columns

Index(['Numero_Transazioni', 'Indirizzo_Sede_legale', 'Forma_Societaria',
       'Comune_Sede_legale', 'Ragione_Sociale', 'Numero_Aggiudicazioni',
       '#Partita_Iva', 'Provincia_Sede_legale', 'Numero_Abilitazioni',
       'Nazione_Sede_legale', 'Regione_Sede_legale',
       'Numero_Contratti_attivi'],
      dtype='object')